<a href="https://colab.research.google.com/github/InowaR/robot/blob/main/radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
from google.colab import output

field = np.zeros((16, 16))
field = np.pad(field, pad_width=1, mode='constant', constant_values=1)

def obstacle(x, y, len):
  i = y
  while i < len + y:
    field[x][i] = 1
    i += 1
  return field

obstacle(4,8,9)
obstacle(9,1,9)
obstacle(13,8,9)

field[15][8] = 2
print(field)
print("Enter \"q\" to exit")

def radar(x, y):
  l = 0    #
  m = 0    #   radar distances
  r = 0    #
  i = x    
  j = y    
  while field[i][j] != 1:
    l += 1
    i -= 1
    j -= 1
  i = x    
  j = y    
  while field[i][j] != 1:
    m += 1
    i -= 1
  i = x    
  j = y    
  while field[i][j] != 1:
    r += 1
    i -= 1
    j += 1
  return [l, m, r]

radar_array = []
button_array = []

def play(x, y):
  i = x    
  j = y
  while True:
    button = input()
    output.clear()
    if button == "w":
      print(field)
      if field[i-1][j] == 1:
        radar(i, j)
        continue  
      field[i-1][j] = 2
      button_array.append([0,1,0])
      output.clear()
      print(field)
      i -= 1
      radar_array.append(radar(i, j))
    elif button == "a":
      print(field)
      if field[i][j-1] == 1:
        radar(i, j)
        continue  
      field[i][j-1] = 2
      button_array.append([1,0,0])
      output.clear()
      print(field)
      j -= 1
      radar_array.append(radar(i, j))
    elif button == "s":
      print(field)
      if field[i+1][j] == 1:
        radar(i, j)
        continue  
      field[i+1][j] = 2
      button_array.append([0,0,0])
      output.clear()
      print(field)
      i += 1
      radar_array.append(radar(i, j))
    elif button == "d":
      print(field)
      if field[i][j+1] == 1:
        radar(i, j)
        continue           
      field[i][j+1] = 2
      button_array.append([0,0,1])
      output.clear()
      print(field)
      j += 1
      radar_array.append(radar(i, j))
    else:
      print(field)
      radar(i, j)
      print("Exit map")
      break

play(15, 8)
print(radar_array)
print(button_array)

def train(X, Y):
  learning_rate = 1
  w1 = np.random.random((3,10))
  w2 = np.random.random((10,3)) 
  w3 = np.random.random((3,3))
  w4 = np.random.random((3,3))

  for i in range(60000):
    S1 = 1 / (1 + np.exp(-(np.dot(X, w1))))
    S2 = 1 / (1 + np.exp(-(np.dot(S1, w2))))
    S3 = 1 / (1 + np.exp(-(np.dot(S2, w3))))
    S4 = 1 / (1 + np.exp(-(np.dot(S3, w4))))
    S4_delta = (Y - S4) * (S4 * (1 - S4))
    S3_delta = S4_delta.dot(w4.T) * (S3 * (1 - S3))
    S2_delta = S3_delta.dot(w3.T) * (S2 * (1 - S2))
    S1_delta = S2_delta.dot(w2.T) * (S1 * (1 - S1))
    w4 += learning_rate * S3.T.dot(S4_delta)
    w3 += learning_rate * S2.T.dot(S3_delta)
    w2 += learning_rate * S1.T.dot(S2_delta)
    w1 += learning_rate * X.T.dot(S1_delta)
  return w1, w2, w3, w4

def predict(test, w1, w2, w3, w4):
  S1 = 1 / (1 + np.exp(-(np.dot(test, w1))))
  S2 = 1 / (1 + np.exp(-(np.dot(S1, w2))))
  S3 = 1 / (1 + np.exp(-(np.dot(S2, w3))))
  S4 = 1 / (1 + np.exp(-(np.dot(S3, w4))))
  return S4

def main():
  a = train(X, Y)
  w1, w2, w3, w4 = a
  b = predict(test, w1, w2, w3, w4)
  print(np.round(b,2))

if __name__ == "__main__":
  X = np.array(radar_array)
  Y = np.array(button_array)
  test = np.array([[5,1,1]])
  main()

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 2. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 2. 2. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.